In [1]:
import os, socket
from pyspark.sql import SparkSession


import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("../src/")
%pwd

'/home/jovyan/nfs-home/scalable_ml_pipelines/src'

In [3]:
import socket

LOCAL_IP = socket.gethostbyname(socket.gethostname())

In [4]:
name_space = "eabraham-373705"

# Master node
kubernetes_master_url = "k8s://https://10.32.7.103:6443"

# Resource settings
driver_cores = "8"
executor_cores = "8"
driver_memory = "30g"
executor_memory = "30g"
executor_memory_overhead = "2g"

# These are the limits
cpu_limit = "3"  # 12 cores
memory_limit = "32g"  # Upto 32 GB
executor_limit = "8"

In [5]:
from pyspark.sql import SparkSession

APP_NAME = 'scalables_executor'


spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master(kubernetes_master_url)\
    .config("spark.driver.host", LOCAL_IP)\
    .config("spark.driver.bindAddress", "0.0.0.0")\
    .config("spark.executor.instances", "2")\
    .config("spark.executor.cores", executor_cores)\
    .config("spark.executor.memory", executor_memory)\
    .config("spark.memory.fraction", "0.7")\
    .config("spark.memory.storageFraction", "0.3")\
    .config("spark.kubernetes.executor.limit.cores", executor_limit)\
    .config("spark.kubernetes.namespace", name_space)\
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")\
    .config("spark.kubernetes.driver.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.deleteOnTermination", "false") \
    .config("spark.kubernetes.container.image.pullPolicy", "Always") \
    .config("spark.kubernetes.container.image", "node03.st:5000/pyspark-hdfs-jupyter:eabraham-373705-v4-executor")\
    .config("spark.local.dir", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .getOrCreate()


23/12/15 14:05:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/15 14:05:18 WARN spark.SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [6]:
spark

In [7]:
from us_used_cars_ml_pipeline.config.configuration import ConfigurationManager
from us_used_cars_ml_pipeline.components.data_cleaning import CleanData

[2023-12-15 14:05:25,640: 145: numexpr.utils: INFO: utils:  Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2023-12-15 14:05:25,641: 157: numexpr.utils: INFO: utils:  NumExpr defaulting to 8 threads.]


In [8]:
config_manager = ConfigurationManager()
data_cleaning_config = config_manager.get_clean_data_config()

data_cleaning = CleanData(data_cleaning_config)

data_cleaning_config

[2023-12-15 14:05:26,441: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/config/config.yaml loaded successfully]
[2023-12-15 14:05:26,444: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/params.yaml loaded successfully]
[2023-12-15 14:05:26,447: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/schema.yaml loaded successfully]


CleanDataConfig(path_to_cleaned_data='hdfs:///home/eabraham-373705/data/cleaned/', path_to_raw_data='hdfs:///home/eabraham-373705/data/raw/raw_data.parquet', features_to_encode='us_used_cars_ml_pipeline/constants/features_to_encode.yaml', features_with_nans='us_used_cars_ml_pipeline/constants/features_with_nans.yaml', features_to_drop='us_used_cars_ml_pipeline/constants/features_to_drop.yaml', glove_model='us_used_cars_ml_pipeline/models/glove/glove-twitter-25.bin', data_types='us_used_cars_ml_pipeline/constants/data_types.yaml', popular_options='us_used_cars_ml_pipeline/constants/popular_options.yaml', label_encodings='us_used_cars_ml_pipeline/constants/label_encodings', kfold_encodings='hdfs:///home/eabraham-373705/data/encodings/kfold_encodings', rare_classes='us_used_cars_ml_pipeline/constants/rare_classes.yaml', seed=42, n_folds=5, rare_classes_count=BoxList([1, 2, 3, 4, 5, 6, 7]))

In [9]:
%%time
data_cleaning.run_stage(spark, is_new_data=False)

[2023-12-15 14:05:44,914: 821: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  Raw data has been read]
[2023-12-15 14:05:44,915: 824: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  PART I. STARTING]
[2023-12-15 14:05:44,921: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/data_types.yaml loaded successfully]
[2023-12-15 14:05:44,928: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/features_with_nans.yaml loaded successfully]
[2023-12-15 14:05:44,931: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/popular_options.yaml loaded successfully]
[2023-12-15 14:05:44,935: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/features_to_encode.yaml loaded successfully]
[2023-12-15 14:05:45,147: 624: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  1. Rows with nans in specified colu

  0%|          | 0/43 [00:00<?, ?it/s]

[2023-12-15 14:05:46,146: 645: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  3. Values in specified columns have been converted]
[2023-12-15 14:05:46,316: 650: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  4. Features power and torque have been splitted]
[2023-12-15 14:05:46,849: 654: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  5. Popular options features have been extracted]


  0%|          | 0/11 [00:00<?, ?it/s]

[2023-12-15 14:05:53,490: 109: us_used_cars_ml_pipeline_logger: INFO: common:  json file saved at: us_used_cars_ml_pipeline/constants/label_encodings/body_type.json]


...

[2023-12-15 14:11:43,296: 512: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.5. Average encodings have been saved]
[2023-12-15 14:11:43,391: 497: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.1. Rare classes have been replaced]
[2023-12-15 14:11:43,490: 500: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.2. Encodings have been read]
[2023-12-15 14:11:43,523: 503: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.3. make_name column has been encoded with encodings]
[2023-12-15 14:11:43,579: 509: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.4. Average encodings have been calculated]


[2023-12-15 14:11:53,002: 512: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.5. Average encodings have been saved]


[2023-12-15 14:11:53,090: 497: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.1. Rare classes have been replaced]
[2023-12-15 14:11:53,173: 500: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.2. Encodings have been read]
[2023-12-15 14:11:53,206: 503: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.3. transmission_display column has been encoded with encodings]
[2023-12-15 14:11:53,261: 509: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.4. Average encodings have been calculated]


[2023-12-15 14:12:04,773: 512: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  8.5. Average encodings have been saved]
[2023-12-15 14:12:04,802: 809: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  3. Specified features have been encoded using k-fold mean target encoder]


[2023-12-15 14:12:19,105: 840: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  PART III. COMPLETED]
CPU times: user 9.2 s, sys: 2.05 s, total: 11.3 s
Wall time: 6min 38s
